In [142]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')


def pos_def(n,x=5):
    A = np.random.rand(n,n)
    return (A+A.T) + x*np.eye(n)

n = 100
A = pos_def(n,x=8)
x_sol = np.floor(np.random.rand(n) * 100)
b = np.dot(A, x_sol)
print np.linalg.cond(A)
print np.linalg.eig(A)[0]

527.774364779
[ 108.29345314   15.79366964    0.20518892    0.29347405    0.52359609
   15.1530344    15.00731674   14.76869942    0.94034136   14.69266744
    1.23382906    1.48534985   14.22850573   14.0955556    13.91983823
    1.66248736    1.97359638    2.01630971    2.16499129    2.30848921
   13.47222696    2.60579897    2.80971585    2.91363142   13.3361485
   13.17135298   13.13490855   13.02297751    3.05324288   12.79986851
   12.66214383    3.14843139    3.23169843   12.55541249    3.36397301
   12.43557426    3.51807342    3.65896964    4.03148012    4.084899
   12.27202443   12.0738117    12.02111584   11.9625255    11.760994
    4.2155311     4.38972775   11.6335055    11.45802032   11.40662085
    4.73596888    4.70514913   11.13139737    4.91100981   11.00077345
    5.08972899   10.96501797    5.26163124    5.36264022    5.4760309
   10.62611031   10.57070028   10.50001115   10.41568219   10.33992713
    5.68872851    5.82397991    5.96456371    9.96574808   10.1392762

In [160]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def prec_gmres(A_0,b_0,alpha,it=100,tol=1e-6,left = True):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = A + alpha*np.identity(b.shape[0])
    
    # Left preconditioner -> A and b changes
    if left:
        b = gmres(M,b,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if left:
            w = gmres(M, np.dot(A,Q[:,i]),tol=tol)[0]
        else:
            w = np.dot(A, gmres(M, Q[:,i],tol=tol)[0])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left:
        return x_tild, i+1
    else:
        return gmres(M, x_tild,tol=tol)[0],i+1

In [161]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(0, b, N/2) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += 2*myfun(i).real
    int_val = 2*myfun(a).real + 2*int_val + 2*myfun(0)# + myfun(b)
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,alpha,v,f, tol=1e-6):
    centro = (l + L)/2.
    radio = (L - l)/2.
    fz = f(z(t, centro, radio))
    dzz = dz(t, radio)
    p = fz*dzz
    # Separamos las matrices
    M = (centro + dzz - alpha)*np.identity(v.shape[0]) - A
    a = M.real
    b = M.imag
    al = v.real
    bet = v.imag
    # Construímos el nuevo sistema Hx = r
    H = np.zeros((a.shape[0]*2, a.shape[1]*2))
    H[:a.shape[0],:a.shape[1]] = a
    H[a.shape[0]:,a.shape[1]:] = a
    H[:a.shape[0],a.shape[1]:] = -b
    H[a.shape[0]:,:a.shape[1]] = b
    r = np.zeros((al.shape[0]*2))
    r[:al.shape[0]] = al
    r[al.shape[0]:] = bet
    
    sol = gmres(H,r,tol=tol)[0]
    gmr = sol[:(sol.shape[0]/2)] + 1j*sol[(sol.shape[0]/2):]
    #gmr = p*spy.gmres((centro + dzz - alpha)*np.identity(v.shape[0]) - A, v)[0]
    return p*gmr


def cauchy_integral(l, L, alpha, v, Nf = 50,N=128,tol=1e-6):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,alpha,v,f,tol=tol)
    val = trapezoid2(g1, N, -np.pi, np.pi) / (2.*np.pi)
    return val

In [165]:
# GMRes using contour integral to compute the preconditioner
def gmres(A_0, b_0, it=100, tol=1e-6, prec=False, left=True, alpha=0.0,aux_tol=1e-6):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    
    if prec:
        # Cambiar a cotas
        lamb, vect = np.linalg.eig(A)
        L = np.amax(np.real(lamb))
        l = np.amin(np.real(lamb))
        d = np.amax(np.imag(lamb))
        l = l + alpha - l/2
        L = L + alpha + L/2
        if left:
            b = cauchy_integral(l, L, alpha, b,tol=aux_tol)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if prec:
            if left:
                w = cauchy_integral(l,L,alpha,np.dot(A,Q[:,i]),tol=aux_tol)
            else:
                w = np.dot(A, cauchy_integral(l, L, alpha, Q[:,i],tol=aux_tol))
        else:
            w = np.dot(A,Q[:,i])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left or not prec:
        return x_tild,i+1
    else:
        return cauchy_integral(l, L, alpha, x_tild,tol=aux_tol),i+1

In [166]:
alpha = 0.05
print "Exact solution:\n", x_sol
print "\nNp Solve:\n", spy.gmres(A,b)
print "\nNormal GMRes:\n", gmres(A,b)
print "\nLeft Prec NC:\n", prec_gmres(A,b,alpha)
#print "\nRight Prec NC:\n", prec_gmres(A,b,alpha,left=False)
print "\nCauchy Left Prec:\n", gmres(A, b,prec=True,alpha=alpha)
#print "\nCauchy Right Prec:\n", gmres(A,b,prec=True,left=False,alpha=alpha)

Exact solution:
[ 90.  16.  29.  93.  80.  22.  83.  50.  11.  70.  32.  29.  22.  11.  97.
  89.  94.  75.  94.  60.  47.  57.  85.  75.  16.  38.  78.  53.  44.   7.
  29.  56.   8.  73.  40.  84.  32.  26.  82.  87.  48.   9.   8.  25.  10.
  24.  32.  12.  33.  52.  77.  89.  50.  47.  30.  43.  26.  18.  39.  56.
  86.   1.  20.  29.  24.  18.  76.  60.  15.  27.  22.  73.  78.  34.  55.
  65.  98.   0.  64.  51.  75.  68.   3.  15.  69.  32.   8.  34.  96.  33.
  55.   7.  46.  51.  15.  49.   9.  15.  36.  48.]

Np Solve:
(array([  8.99878118e+01,   1.59811360e+01,   2.89745541e+01,
         9.30733074e+01,   8.01131188e+01,   2.20237210e+01,
         8.29754940e+01,   5.00895140e+01,   1.11430401e+01,
         6.99883650e+01,   3.21802827e+01,   2.89792113e+01,
         2.22251489e+01,   1.10762320e+01,   9.70596187e+01,
         8.91743992e+01,   9.38412722e+01,   7.50127570e+01,
         9.40646965e+01,   5.99642453e+01,   4.70254094e+01,
         5.68989207e+01,   8.48720310